## 05 · Manual Restoration from Checkpoints  

If the maximum number of retries is reached, you can still **manually restore training** by creating a new `TorchTrainer` with the same configuration:  

- Use the same `train_loop_ray_train_with_checkpoint_loading` so the loop can resume from a checkpoint.  
- Provide the same `run_config` (name, storage path, and failure config).  
- Pass in the same dataset and scaling configuration.  

Ray Train will detect the latest checkpoint in the specified `storage_path` and resume training from that point.  

In [ ]:
# 05. Manually restore a trainer from the last checkpoint

restored_trainer = TorchTrainer(
    train_loop_per_worker=train_loop_ray_train_with_checkpoint_loading,  # loop supports checkpoint loading
        train_loop_config={   # hyperparameters must match
        "num_epochs": 1,
        "global_batch_size": 512,
    },
    scaling_config=scaling_config,  # same resource setup as before
    run_config=ray.train.RunConfig(
        name="fault-tolerant-cifar-vit",  # must match previous run name
        storage_path=storage_path,       # path where checkpoints are saved
        failure_config=failure_config,   # still allow retries
    ),
    datasets=datasets,  # same dataset as before
)

### 06 · Resume Training from the Last Checkpoint  

Calling `restored_trainer.fit()` will continue training from the most recent checkpoint found in the specified storage path.  

- If all epochs were already completed in the previous run, the trainer will terminate immediately.  
- If training was interrupted mid-run, it will resume from the saved epoch, restoring both the **model** and **optimizer** state.  
- The returned `Result` object confirms that training picked up correctly and contains metrics, checkpoints, and logs.  

In [ ]:
# 06. Resume training from the last checkpoint

# Fit the restored trainer → continues from last saved epoch
# If all epochs are already complete, training ends immediately
result = restored_trainer.fit()

# Display final training results (metrics, checkpoints, etc.)
result